# Elo for csgo teams

In [180]:
import pandas as pd
import math

In [181]:
match_online = pd.read_csv('data/matches_Online_21.csv',sep=';')
match_online['event_tier'] = 'B'
match_lan = pd.read_csv('data/matches_Lan_21.csv',sep=';')
match_big_events = pd.read_csv('data/matches_BigEvents_21.csv',sep=';')
match_majors = pd.read_csv('data/matches__Majors_21.csv',sep=';')
match_history  = pd.concat([match_majors,match_big_events,match_lan,match_online])
match_history.drop("Unnamed: 0",axis=1,inplace=True)

In [182]:
match_history.head(50)

,Date,Team1,Team2,Map,Event,event_type,event_tier
0,30/11/13,fnatic (16),Ninjas in Pyjamas (2),trn Train,DreamHack Winter 2013,Majors,S
1,30/11/13,Ninjas in Pyjamas (16),fnatic (6),inf Inferno,DreamHack Winter 2013,Majors,S
2,30/11/13,fnatic (16),Ninjas in Pyjamas (14),d2 Dust2,DreamHack Winter 2013,Majors,S
3,30/11/13,Complexity (7),fnatic (16),mrg Mirage,DreamHack Winter 2013,Majors,S
4,30/11/13,fnatic (16),Complexity (7),inf Inferno,DreamHack Winter 2013,Majors,S
5,30/11/13,Ninjas in Pyjamas (16),VeryGames (5),nuke Nuke,DreamHack Winter 2013,Majors,S
6,30/11/13,VeryGames (16),Ninjas in Pyjamas (6),inf Inferno,DreamHack Winter 2013,Majors,S
7,30/11/13,Ninjas in Pyjamas (16),VeryGames (13),d2 Dust2,DreamHack Winter 2013,Majors,S
8,29/11/13,Astana Dragons (12),Complexity (16),inf Inferno,DreamHack Winter 2013,Majors,S
9,29/11/13,Complexity (16),Astana Dragons (7),d2 Dust2,DreamHack Winter 2013,Majors,S


## Clean data

In [183]:
#renaming collumns
match_history.rename(columns={'Date':'date', 'Team1':"team1", 'Team2':"team2", 'Map':"map", 'Event':"event"},inplace=True)
#separating score and name to get a clean data
match_history[['team1','s1']] = match_history['team1'].str.split('(',1,expand=True)
match_history[['team2','s2']] = match_history['team2'].str.split('(',1,expand=True)
match_history['team1'] = match_history['team1'].str.strip()
match_history['team2'] = match_history['team2'].str.strip()
match_history['s1'] = match_history['s1'].str[:-1]
match_history['s2'] = match_history['s2'].str[:-1]
#separating the abreviation of the map
match_history['map'] = match_history['map'].str[4:]
# Score were not properly score
match_history.loc[match_history['s2'] == "-",'s2'] = 16

### Removing duplicates

In [184]:
# priortyze removing Online>Lan>BigEvents>Majors
match_history.sort_values(by=['event_tier','event_type'],inplace = True)
match_history.drop_duplicates(subset=(match_history.columns.difference(['event_tier','event_type'])),keep='last',inplace=True)


## Change type

### Score

In [185]:
match_history['s1'] = pd.to_numeric(match_history['s1'])
match_history['s2'] = pd.to_numeric(match_history['s2'])

### date

In [186]:
match_history['date'] = pd.to_datetime(match_history['date'])


## Adding information for the table

## Winner

In [187]:
match_history['winner'] = (match_history['s1'] - match_history['s2']>0)

## Elo collumn

In [188]:
match_history[['elo1','elo2']] = pd.Series([round(1500),round(1500)])    


## Index

In [189]:
match_history.sort_values(by=["date"],inplace=True)
match_history.reset_index(drop=True,inplace=True)
match_history['index1'] = match_history.index

# Sorting by date

In [190]:
match_history.sort_values(by='date',inplace=True)

# Exporting

In [193]:
match_history.to_csv('data/matches.csv',sep=';',index=False)

# Test

In [191]:
astralis = match_history.query('team1 == "Astralis"')

In [192]:
astralis

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
12680,2016-01-04,Astralis,fnatic,Overpass,MLG Columbus 2016,Majors,S,16,10,True,1500,1500,12680
12884,2016-01-10,Astralis,SK,Overpass,ESL One New York 2016,BigEvents,S,13,16,False,1500,1500,12884
12853,2016-01-10,Astralis,OpTic,Overpass,ESL One New York 2016,BigEvents,S,12,16,False,1500,1500,12853
13125,2016-01-22,Astralis,MOUZ,Mirage,DreamHack ZOWIE Open Leipzig 2016,Lan,A,22,19,True,1500,1500,13125
13152,2016-01-23,Astralis,Luminosity,Train,DreamHack ZOWIE Open Leipzig 2016,Lan,A,8,16,False,1500,1500,13152
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103351,2021-11-27,Astralis,Heroic,Nuke,BLAST Premier Fall Final 2021,BigEvents,S,16,4,True,1500,1500,103351
103400,2021-11-28,Astralis,Vitality,Inferno,BLAST Premier Fall Final 2021,BigEvents,S,11,16,False,1500,1500,103400
104164,2021-12-15,Astralis,Vitality,Mirage,BLAST Premier World Final 2021,BigEvents,S,6,16,False,1500,1500,104164
104226,2021-12-16,Astralis,Ninjas in Pyjamas,Inferno,BLAST Premier World Final 2021,BigEvents,S,16,8,True,1500,1500,104226
